In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import csv

In [ ]:
movie_lines_df = pd.read_csv('movie_lines.txt', sep=' \+\+\+\$\+\+\+ ', encoding='unicode_escape', names=['lineID','charID','movieID','char_name','utter'])

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [ ]:
conv_df = movie_lines_df[:10000].dropna()

In [ ]:
# создание сортированного словаря по возрастанию с парами (номер фразы: фраза)
def get_all_sorted_chats(all_conversations):
    all_chats = {}
    for tokens in all_conversations.values:
        all_chats[int(tokens[0][1:])] = tokens[4]
    return sorted(all_chats.items(), key=lambda x: x[0])

In [ ]:
# Создание словаря с попарными диалогами
def get_conversation_dict(sorted_chats):
  conv_dict = {}
  counter = 1
  conv_ids = [] # список текущего диалога
  for i in range(1, len(sorted_chats) + 1):
    if i < len(sorted_chats):
        # если номер текущей строки отличается от предыдущего на 1
        # то эта строка является частью текущего диалога
      if (sorted_chats[i][0] - sorted_chats[i - 1][0]) == 1:
          # если предыдущая строка не была добавлена ​​ранее, 
          # то мы должны добавить ее сейчас
        if sorted_chats[i - 1][1] not in conv_ids:
          conv_ids.append(sorted_chats[i - 1][1])
        conv_ids.append(sorted_chats[i][1]) # или просто добавьте текущую строку
            # а если разница больше 1 - значит новый разговор начался 
            # и надо очистить conv_ids
      elif (sorted_chats[i][0] - sorted_chats[i - 1][0]) > 1:
        conv_dict[counter] = conv_ids
        conv_ids = []
        counter += 1
      else: continue
  return conv_dict

In [ ]:
import re
# Форматирование и очистка текста
def clean_text(text_to_clean):
    res = text_to_clean.lower()
    res = re.sub(r"i'm", "i am", res)
    res = re.sub(r"he's", "he is", res)
    res = re.sub(r"she's", "she is", res)
    res = re.sub(r"it's", "it is", res)
    res = re.sub(r"that's", "that is", res)
    res = re.sub(r"what's", "what is", res)
    res = re.sub(r"where's", "where is", res)
    res = re.sub(r"how's", "how is", res)
    res = re.sub(r"\'ll", " will", res)
    res = re.sub(r"\'ve", " have", res)
    res = re.sub(r"\'re", " are", res)
    res = re.sub(r"\'d", " would", res)
    res = re.sub(r"\'re", " are", res)
    res = re.sub(r"won't", "will not", res)
    res = re.sub(r"can't", "cannot", res)
    res = re.sub(r"n't", " not", res)
    res = re.sub(r"n'", "ng", res)
    res = re.sub(r"'bout", "about", res)
    res = re.sub(r"'til", "until", res)
    res = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", res)
    return res

In [ ]:
def get_clean_q_and_a(conversations_dictionary):
    ctx_and_target = []
    for current_conv in conversations_dictionary.values():
        # проверка диалога на четное количество строк
        if len(current_conv) % 2 != 0:
            current_conv = current_conv[:-1]
        # создание кортежей с вопросами и ответами
        for i in range(0, len(current_conv), 2):
            ctx_and_target.append((current_conv[i], current_conv[i + 1]))
    # с пмощью zip с оператором * 
    # распаковываем кортежи в 2 независимых списка вопросов и ответов
    context, target = zip(*ctx_and_target)
    context_dirty = list(context)
    target_dirty = list(target)

    # очищаем вопросы от сокращенных форм, 
    # небуквенных символов и преобразовать их в нижний регистр
    clean_questions = list()
    for i in range(len(context_dirty)):
        clean_questions.append(clean_text(context_dirty[i]))

    # делаем то же самое с ответами
    # и добаляем токены старта и финиша ответа
    clean_answers = list()
    for i in range(len(target_dirty)):
        clean_answers.append('<START> '+ clean_text(target_dirty[i]) + ' <END>')
        
    return clean_questions, clean_answers

In [ ]:
all_conv = get_all_sorted_chats(conv_df)
conv_dict = get_conversation_dict(all_conv)
df_m_1, df_f_1 = get_clean_q_and_a(conv_dict)

In [ ]:
from keras_preprocessing.text import Tokenizer

target_regex = '!"#$%&()*+,-./:;=?@[\]^_`{|}~\t\n\'0123456789'
# Tokenizer позволяет векторизовать наш корпус, превращая каждое предложение
# в последовательность целых чисел, где каждое целое число является индексом
# токена во внутреннем словаре
tokenizer = Tokenizer(filters=target_regex)
tokenizer.fit_on_texts(df_m_1 + df_f_1)
VOCAB_SIZE = len(tokenizer.word_index) + 1
print('Размер словаря : {}'.format(VOCAB_SIZE))

Vocabulary size : 7940


In [ ]:
tokenized_questions = tokenizer.texts_to_sequences(df_m_1)
maxlen_questions = max([len(x) for x in tokenized_questions]) # максимальная длина вопроса

# делаем паддинг для всех вопросов которые короче самого длинного вопроса
encoder_input_data = tf.keras.utils.pad_sequences(tokenized_questions, 
                                 maxlen=maxlen_questions,
                                 padding='post')
print(f'Количество вопросов: {encoder_input_data.shape[0]} Длина вопросов: {encoder_input_data.shape[1]}')

tokenized_answers = tokenizer.texts_to_sequences(df_f_1)
maxlen_answers = max([len(x) for x in tokenized_answers]) # максимальная длина ответа

# делаем паддинг для всех ответов которые короче самого длинного вопроса
decoder_input_data = tf.keras.utils.pad_sequences(tokenized_answers,   
                                   maxlen=maxlen_answers,
                                   padding='post')
print(f'Количество ответов: {decoder_input_data.shape[0]} Длина ответов: {decoder_input_data.shape[1]}')

(4707, 223)
(4707, 132)


In [ ]:
# Убрать <start> в начале ответов
for i in range(len(tokenized_answers)):
    tokenized_answers[i] = tokenized_answers[i][1:]
decoder_output = tf.keras.utils.pad_sequences(tokenized_answers, maxlen=maxlen_answers, padding='post')

In [ ]:
H_SIZE = 256 # Размерность скрытого состояния LSTM
EMB_SIZE = 256 # размерность эмбеддингов (и для входных и для выходных цепочек)

# Encoder
enc_inputs = tf.keras.layers.Input(shape=(None,))
enc_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, EMB_SIZE, mask_zero=True)(enc_inputs)

enc_outputs, state_h, state_c = tf.keras.layers.LSTM(H_SIZE, return_sequences=False, return_state=True)(enc_embedding)
enc_states = [state_h, state_c]

# Decoder
dec_inputs = tf.keras.layers.Input(shape=(None,))
dec_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, EMB_SIZE, mask_zero=True)(dec_inputs)

dec_lstm = tf.keras.layers.LSTM(H_SIZE, return_state=True, return_sequences=True)
dec_outputs, _, _ = dec_lstm(dec_embedding, initial_state=enc_states)

dec_dense = tf.keras.layers.Dense(VOCAB_SIZE, activation='softmax')
output = dec_dense(dec_outputs)

#Создание модели для обучения
model = tf.keras.Model([enc_inputs, dec_inputs], output)

In [ ]:
loss = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer='rmsprop', loss=loss)
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_19 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 input_20 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 embedding_6 (Embedding)        (None, None, 256)    2032640     ['input_19[0][0]']               
                                                                                                  
 embedding_7 (Embedding)        (None, None, 256)    2032640     ['input_20[0][0]']               
                                                                                            

In [ ]:
model.fit([encoder_input_data, decoder_input_data], 
          decoder_output, 
          batch_size=64, 
          epochs=150)

Epoch 1/150
74/74 [==============================] - 12s 65ms/step - loss: 0.5692
Epoch 2/150
74/74 [==============================] - 5s 63ms/step - loss: 0.5053
Epoch 3/150
74/74 [==============================] - 5s 66ms/step - loss: 0.4855
Epoch 4/150
74/74 [==============================] - 5s 67ms/step - loss: 0.4707
Epoch 5/150
74/74 [==============================] - 5s 67ms/step - loss: 0.4590
Epoch 6/150
74/74 [==============================] - 5s 68ms/step - loss: 0.4480
Epoch 7/150
74/74 [==============================] - 5s 64ms/step - loss: 0.4378
Epoch 8/150
74/74 [==============================] - 5s 65ms/step - loss: 0.4290
Epoch 9/150
74/74 [==============================] - 5s 64ms/step - loss: 0.4211
Epoch 10/150
74/74 [==============================] - 5s 64ms/step - loss: 0.4132
Epoch 11/150
74/74 [==============================] - 5s 64ms/step - loss: 0.4060
Epoch 12/150
74/74 [==============================] - 5s 65ms/step - loss: 0.3988
Epoch 13/150
74/74 [====

In [ ]:
model.save('model_chat.h5') # сохраниение обученной модели

In [ ]:
#Функция конвертирования строки в токены
def str_to_tokens(inp_str: str):
    words = inp_str.lower().split()
    tokens_list = list()
    for current_word in words:
        result = tokenizer.word_index.get(current_word, '')
        if result != '':
            tokens_list.append(result)
    return tf.keras.utils.pad_sequences([tokens_list],
                         maxlen=maxlen_questions,
                         padding='post')

In [ ]:
# Создание моделей для инференса
def make_inference_models():
    dec_state_input_h = tf.keras.layers.Input(shape=(H_SIZE,))
    dec_state_input_c = tf.keras.layers.Input(shape=(H_SIZE,))
    dec_states_inputs = [dec_state_input_h, dec_state_input_c]

    dec_outputs, state_h, state_c = dec_lstm(dec_embedding, initial_state=dec_states_inputs)
    dec_states = [state_h, state_c]

    dec_outputs = dec_dense(dec_outputs)

    dec_model = tf.keras.Model(
        inputs=[dec_inputs, dec_states_inputs],
        outputs=[dec_outputs, dec_states])
    print('Inference decoder:')
    dec_model.summary()

    print('Inference encoder:')
    enc_model = tf.keras.Model(inputs=enc_inputs, outputs=enc_states)
    enc_model.summary()

    return enc_model, dec_model

enc_model, dec_model = make_inference_models()

Inference decoder:
Model: "model_15"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_20 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 embedding_7 (Embedding)        (None, None, 256)    2032640     ['input_20[0][0]']               
                                                                                                  
 input_61 (InputLayer)          [(None, 256)]        0           []                               
                                                                                                  
 input_62 (InputLayer)          [(None, 256)]        0           []                               
                                                                        

In [ ]:
# Инференс
for _ in range(10):
    states_values = enc_model(str_to_tokens(input('Enter question : ')))
    
    empty_target_seq = np.zeros((1, 1))
    empty_target_seq[0, 0] = tokenizer.word_index.get('start')
    stop_condition = False
    decoded_translation = ''

    while not stop_condition:
        pred_input = [empty_target_seq] + states_values
        dec_outputs, dec_states = dec_model(pred_input)
        sampled_word_index = np.argmax(dec_outputs[0, -1, :])
        sampled_word = tokenizer.index_word.get(sampled_word_index)
        
        if sampled_word != '<end>':
          decoded_translation += ' {}'.format(sampled_word)
        
        if sampled_word == '<end>' or len(decoded_translation.split()) > maxlen_answers:
                stop_condition = True

        empty_target_seq = np.zeros((1, 1))
        empty_target_seq[0, 0] = sampled_word_index
        states_values = dec_states

    print(decoded_translation)

Enter question : hekko
1/1 [==============================] - 0s 21ms/step
 with feel the caught the course from the police we have to go i told them
Enter question : руддщ
1/1 [==============================] - 0s 18ms/step
 with feel the caught the course from the police we have to go i told them
Enter question : hello
1/1 [==============================] - 0s 21ms/step
 once in feel the imagine does you know me


KeyboardInterrupt: ignored

In [ ]:
H_SIZE = 256 # Размерность скрытого состояния LSTM
EMB_SIZE = 256 # размерность эмбеддингов (и для входных и для выходных цепочек)

class Encoder(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.embed = tf.keras.layers.Embedding(VOCAB_SIZE, EMB_SIZE, mask_zero=True)
        self.lstm = tf.keras.layers.LSTM(H_SIZE, return_sequences=False, return_state=True)
        
    def call(self, inputs):
        out = self.embed(inputs)
        _, h, c = self.lstm(out)
        state = [h, c]
        return state

class Decoder(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.embed = tf.keras.layers.Embedding(VOCAB_SIZE, EMB_SIZE, mask_zero=True)
        self.lstm = tf.keras.layers.LSTM(H_SIZE, return_sequences=True, return_state=True)
        self.fc = tf.keras.layers.Dense(VOCAB_SIZE, activation='softmax')
        
    def call(self, inputs, init_state):
        out = self.embed(inputs)
        out, h, c = self.lstm(out, initial_state=init_state)
        out = self.fc(out)
        state = [h, c]
        return out, state

In [ ]:
encoder_model = Encoder()
decoder_model = Decoder()

encoder_inputs = tf.keras.layers.Input(shape=(None,))
decoder_inputs = tf.keras.layers.Input(shape=(None,))

enc_state = encoder_model(encoder_inputs)
decoder_outputs, _ = decoder_model(decoder_inputs, enc_state)

seq2seq = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
loss = tf.losses.SparseCategoricalCrossentropy()
seq2seq.compile(optimizer='rmsprop', loss=loss)
seq2seq.summary()

Model: "model_27"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_90 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 input_91 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 encoder_15 (Encoder)           [(None, 256),        2557952     ['input_90[0][0]']               
                                 (None, 256)]                                                     
                                                                                                  
 decoder_15 (Decoder)           ((None, None, 7940)  4598532     ['input_91[0][0]',        

In [ ]:
BATCH_SIZE = 64
EPOCHS = 150

seq2seq.fit([encoder_input_data, decoder_input_data],
            decoder_output,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS)

Epoch 1/150
74/74 [==============================] - 5s 66ms/step - loss: 0.1271
Epoch 2/150
74/74 [==============================] - 5s 66ms/step - loss: 0.1213
Epoch 3/150
74/74 [==============================] - 5s 66ms/step - loss: 0.1155
Epoch 4/150
74/74 [==============================] - 5s 67ms/step - loss: 0.1103
Epoch 5/150
74/74 [==============================] - 5s 69ms/step - loss: 0.1052
Epoch 6/150
74/74 [==============================] - 5s 71ms/step - loss: 0.1003
Epoch 7/150
74/74 [==============================] - 5s 68ms/step - loss: 0.0954
Epoch 8/150
74/74 [==============================] - 5s 67ms/step - loss: 0.0908
Epoch 9/150
74/74 [==============================] - 5s 67ms/step - loss: 0.0867
Epoch 10/150
74/74 [==============================] - 5s 68ms/step - loss: 0.0817
Epoch 11/150
74/74 [==============================] - 5s 67ms/step - loss: 0.0776
Epoch 12/150
74/74 [==============================] - 5s 71ms/step - loss: 0.0737
Epoch 13/150
74/74 [=====

KeyboardInterrupt: ignored

In [ ]:
seq2seq.save('model_seq2seq_chat.h5')

In [ ]:
# Инференс
for _ in range(10):
    encoder_states = encoder_model(str_to_tokens(input('Enter question : ')))
    
    empty_target_seq = np.zeros((1, 1))
    empty_target_seq[0, 0] = tokenizer.word_index.get('<start>')

    decoded_sentence = ''
    stop_condition = False
    while not stop_condition:
        output_tokens, decoder_states = decoder_model(empty_target_seq, encoder_states)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = tokenizer.index_word.get(sampled_token_index)
        
        if sampled_word != '<end>':
          decoded_sentence += ' {}'.format(sampled_word)

        if (sampled_word == '<end>' or len(decoded_sentence) > maxlen_answers):
            stop_condition = True

        empty_target_seq = np.zeros((1, 1))
        empty_target_seq[0, 0] = sampled_token_index
        encoder_states = decoder_states

    print(decoded_sentence)

Enter question : hello
 mrs mathews it is thomas do you remember i was there a few weeks ago asking about your daughter
Enter question : did you change
 i was at hand it
Enter question : how are you
 i am okay honey i am okay


KeyboardInterrupt: ignored